Looking at Kentucky county-level data to explore connections between child poverty, education outcomes, and early education access.

In [74]:
# import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os 

## Load County Health Rankings (National)
Load the County Health Rankings dataset from the national CSV file.  
Source: County Health Rankings & Roadmaps (https://www.countyhealthrankings.org/)  
Reason: Contains health, education, and economic indicators for all U.S. counties, which will be filtered to Kentucky for this analysis.

In [75]:
# load county health rankings (national)
df_chr_ky = pd.read_csv("../data/analytic_data2025_v2.csv", low_memory=False)

### Filter to Kentucky only
Keep only rows where the state abbreviation is KY.

In [76]:
# filter to Kentucky
df_chr_ky = df[df["State Abbreviation"] == "KY"]

### Select columns needed
Keep only state, county, child poverty percentage, and high school completion percentage.

In [77]:
# keep only the columns needed
columns_to_keep = [
    "State Abbreviation",
    "Name",                           # county name
    "Children in Poverty raw value",
    "High School Completion raw value"
]

df_chr_ky = df_chr_ky[columns_to_keep]
df_chr_ky.head(3)

,State Abbreviation,Name,Children in Poverty raw value,High School Completion raw value
1022,KY,Kentucky,0.202,0.8853320347
1023,KY,Adair County,0.278,0.8578791192
1024,KY,Allen County,0.215,0.8495460441


### Rename columns for clarity
Rename to lowercase, descriptive names to match other datasets.

In [78]:
# rename to match the format of other datasets
df_chr_ky.columns = [
    "state",
    "county",
    "child_poverty_pct",
    "hs_completion_pct"
]

df_chr_ky.head(3)

,state,county,child_poverty_pct,hs_completion_pct
1022,KY,Kentucky,0.202,0.8853320347
1023,KY,Adair County,0.278,0.8578791192
1024,KY,Allen County,0.215,0.8495460441


### Clean county names
Remove " County" from county names and drop the statewide summary row.

In [79]:
df_chr_ky["county"] = df_chr_ky["county"].str.replace(" County", "", regex=False)
df_chr_ky = df_chr_ky[df_chr_ky["county"] != "Kentucky"]
df_chr_ky.head(3)

,state,county,child_poverty_pct,hs_completion_pct
1023,KY,Adair,0.278,0.8578791192
1024,KY,Allen,0.215,0.8495460441
1025,KY,Anderson,0.122,0.9169763817


### Convert columns to numeric
Ensure percentages are stored as numeric types for calculations.

In [80]:
df_chr_ky["child_poverty_pct"] = pd.to_numeric(df_chr_ky["child_poverty_pct"], errors="coerce")
df_chr_ky["hs_completion_pct"] = pd.to_numeric(df_chr_ky["hs_completion_pct"], errors="coerce")
df_chr_ky.describe(include="all").T.head(6)

df_chr_ky.describe(include="all").T.head(6)

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
state,120,1,KY,120,NaN,NaN,NaN,NaN,NaN,NaN,NaN
county,120,120,Adair,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
child_poverty_pct,120.0,NaN,NaN,NaN,0.235808,0.085694,0.05,0.17825,0.2225,0.27925,0.536
hs_completion_pct,120.0,NaN,NaN,NaN,0.851548,0.05681,0.659858,0.813814,0.854216,0.896419,0.943003


Check for missing values

In [81]:
df_chr_ky.isnull().sum()

state                0
county               0
child_poverty_pct    0
hs_completion_pct    0
dtype: int64

## Load and clean childcare subsidies dataset  
Source: Kentucky KIDS COUNT  
I wanted to filter to county-level data, keep only relevant columns, clean names, and convert to numeric types.

In [82]:
# Load childcare subsidies dataset
subsidies_df = pd.read_csv("../data/Children receiving childcare subsidies.csv")
subsidies_df.shape

(2662, 5)

## Clean childcare subsidies data  
Source: Kentucky KIDS COUNT.  
Steps: filter to county rows, keep only needed columns, rename for clarity, clean county names, and convert subsidies to numeric.

In [90]:
# Load and clean subsidies data
subsidies_df = pd.read_csv("../data/Children receiving childcare subsidies.csv")
subsidies_df = subsidies_df[subsidies_df["LocationType"] == "County"]
subsidies_df = subsidies_df[["Location", "TimeFrame", "Data"]]
subsidies_df.columns = ["county", "year", "childcare_subsidies"]
subsidies_df["county"] = subsidies_df["county"].str.replace(" County", "", regex=False)
subsidies_df["childcare_subsidies"] = pd.to_numeric(subsidies_df["childcare_subsidies"], errors="coerce")

# Filter to latest year
latest_year = subsidies_df["year"].max()
subsidies_latest = subsidies_df[subsidies_df["year"] == latest_year].copy()

# Merge CHR data with latest subsidies
merged_df = df_chr_ky.merge(subsidies_latest, on="county", how="left")

print(f"Latest subsidies year: {latest_year}")
print(f"Merged dataset shape: {merged_df.shape}")
merged_df.head()


Latest subsidies year: 2022
Merged dataset shape: (120, 6)


,state,county,child_poverty_pct,hs_completion_pct,year,childcare_subsidies
0,KY,Adair,0.278,0.857879,2022,253.0
1,KY,Allen,0.215,0.849546,2022,86.0
2,KY,Anderson,0.122,0.916976,2022,121.0
3,KY,Ballard,0.224,0.936684,2022,44.0
4,KY,Barren,0.226,0.865345,2022,327.0


In [88]:
# Filter to the latest year
latest_year = subsidies_df["year"].max()
subsidies_latest = subsidies_df[subsidies_df["year"] == latest_year].copy()


# Merge CHR data (ky_df) with latest subsidies data
merged_df = ky_df.merge(subsidies_latest, on="county", how="left")

# Preview merged data
print(f"Latest subsidies year: {latest_year}")
print(f"Merged dataset shape: {merged_df.shape}")
merged_df.head()


KeyError: 'county'

In [ ]:
# Load graduation rates dataset
grad_df = pd.read_csv("../data/Cohort graduation rate.csv")

# Preview columns
print(grad_df.columns)
print(grad_df.head())

# Keep only needed columns (adjust names if different)
# Example assumes columns like: 'Location', 'TimeFrame', 'Data'
grad_df = grad_df[["Location", "TimeFrame", "Data"]]

# Rename for clarity
grad_df.columns = ["county", "year", "grad_rate"]

# Clean county names to match other datasets
grad_df["county"] = grad_df["county"].str.replace(" County", "", regex=False)

# Ensure numeric type for graduation rate
grad_df["grad_rate"] = pd.to_numeric(grad_df["grad_rate"], errors="coerce")

# Filter to the most recent year
latest_grad_year = grad_df["year"].max()
grad_latest = grad_df[grad_df["year"] == latest_grad_year]

# Preview cleaned and filtered graduation data
print(f"Latest graduation year: {latest_grad_year}")
print(grad_latest.shape)
grad_latest.head()


Index(['LocationType', 'Location', 'TimeFrame', 'DataFormat', 'Data'], dtype='object')
  LocationType  Location  TimeFrame DataFormat  Data
0        State  Kentucky       2013       Rate  86.1
1        State  Kentucky       2014       Rate  87.5
2        State  Kentucky       2015       Rate    88
3        State  Kentucky       2016       Rate  88.6
4        State  Kentucky       2017       Rate  89.8
Latest graduation year: 2024
(177, 3)


,county,year,grad_rate
11,Kentucky,2024,92.2
23,Adair Co.,2024,94.9
35,Allen Co.,2024,94.0
47,Anchorage Ind.,2024,NaN
59,Anderson Co.,2024,93.0
